## Unit 14. 模型評估與選擇 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 0. 環境設定

In [ ]:
from pathlib import Path
import tensorflow as tf
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P3_Unit14_Homework'
SOURCE_DATA_DIR = 'catalyst_performance'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_3' / 'Unit14'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")


# ========================================
# 檢查 GPU 狀態
# ========================================
print(f"\nTensorFlow Version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ 偵測到 GPU：{gpus[0].name}")
    print("  （訓練速度將明顯快於僅用 CPU）")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("△ 未偵測到 GPU。")
    print("  訓練速度將使用 CPU（速度較慢但仍可完成）")

---
### 載入相關套件

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn套件
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, validation_curve, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from IPython.display import display

# Bayesian Optimization (需安裝 scikit-optimize)
try:
    from skopt import BayesSearchCV
    from skopt.space import Real, Integer
    print("✓ scikit-optimize 已安裝")
except ImportError:
    print("⚠ 未安裝 scikit-optimize，請執行: pip install scikit-optimize")

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定matplotlib顯示
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✓ 所有套件載入完成")

---
### **I. 練習題: 模型評估與選擇**

### 作業說明

本次作業將使用**催化劑性能數據**，模擬化工廠中催化劑篩選與優化的實際場景。你需要：
1. 訓練至少5種不同類型的模型
2. 進行超參數調整（Grid Search + Bayesian Optimization）
3. 繪製學習曲線與驗證曲線
4. 撰寫模型選擇報告
5. 進行特徵重要性分析

**評分重點**：
- 模型訓練與評估的完整性（30%）
- 超參數調整的方法與結果分析（25%）
- 學習曲線/驗證曲線的繪製與診斷（20%）
- 模型選擇報告的邏輯性與專業性（15%）
- 特徵重要性分析與化學意義解釋（10%）

---
### 1. 數據生成與探索

以下程式碼模擬催化劑性能實驗數據，包含8個特徵：
- **Temperature (°C)**: 反應溫度
- **Pressure (bar)**: 反應壓力
- **Catalyst_Loading (wt%)**: 催化劑負載量
- **Surface_Area (m²/g)**: 催化劑比表面積
- **Pore_Volume (cm³/g)**: 催化劑孔容
- **Active_Site_Density (sites/nm²)**: 活性位點密度
- **H2_Flow_Rate (mL/min)**: 氫氣流速
- **Reactant_Concentration (mol/L)**: 反應物濃度

**目標變數 (Y)**：
- **Conversion (%)**: 轉化率（越高越好）

**數據特性**：
- 1000筆實驗數據
- 包含非線性關係與交互作用
- 添加真實實驗誤差（噪聲）

In [ ]:
# 生成催化劑性能數據
np.random.seed(SEED)
n_samples = 1000

# 生成8個特徵
Temperature = np.random.uniform(200, 400, n_samples)  # °C
Pressure = np.random.uniform(1, 50, n_samples)  # bar
Catalyst_Loading = np.random.uniform(0.5, 5.0, n_samples)  # wt%
Surface_Area = np.random.uniform(50, 500, n_samples)  # m²/g
Pore_Volume = np.random.uniform(0.1, 1.0, n_samples)  # cm³/g
Active_Site_Density = np.random.uniform(1, 20, n_samples)  # sites/nm²
H2_Flow_Rate = np.random.uniform(10, 200, n_samples)  # mL/min
Reactant_Concentration = np.random.uniform(0.1, 2.0, n_samples)  # mol/L

# 生成目標變數（轉化率）
# 包含非線性關係與交互作用
Conversion = (
    20 +  # 基礎轉化率
    0.10 * Temperature +  # 溫度正相關
    -0.00013 * Temperature**2 +  # 溫度二次項（過高會降低選擇性）
    0.20 * Pressure +  # 壓力正相關
    1.5 * Catalyst_Loading +  # 催化劑負載量正相關
    0.015 * Surface_Area +  # 比表面積正相關
    3.5 * Pore_Volume +  # 孔容正相關
    0.35 * Active_Site_Density +  # 活性位點密度正相關
    0.035 * H2_Flow_Rate +  # 氫氣流速正相關
    2.0 * Reactant_Concentration +  # 反應物濃度正相關
    # 交互作用項
    0.002 * Temperature * Catalyst_Loading +  # 溫度與催化劑負載量交互
    0.007 * Pressure * Reactant_Concentration +  # 壓力與濃度交互
    -0.00007 * Surface_Area * Pore_Volume +  # 表面積與孔容交互（負相關）
    # 添加噪聲（真實實驗誤差）
    np.random.normal(0, 5, n_samples)
)

# 限制轉化率在 0-100% 範圍內
Conversion = np.clip(Conversion, 0, 100)

# 建立 DataFrame
df = pd.DataFrame({
    'Temperature': Temperature,
    'Pressure': Pressure,
    'Catalyst_Loading': Catalyst_Loading,
    'Surface_Area': Surface_Area,
    'Pore_Volume': Pore_Volume,
    'Active_Site_Density': Active_Site_Density,
    'H2_Flow_Rate': H2_Flow_Rate,
    'Reactant_Concentration': Reactant_Concentration,
    'Conversion': Conversion
})

print("✓ 催化劑性能數據已生成")
print(f"數據形狀: {df.shape}")
print("\n前5筆數據:")
display(df.head())
print("\n統計摘要:")
display(df.describe())

---
### 2. 數據分割與標準化

**任務說明**：
- 將數據分割為訓練集（80%）與測試集（20%）
- 對特徵進行標準化處理（StandardScaler）
- 注意：**僅使用訓練集的統計量進行標準化**，避免數據洩漏

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 請在此完成數據分割與標準化
# ========================================

# 提示：
# 1. 使用 train_test_split() 分割數據
# 2. 使用 StandardScaler() 進行標準化
# 3. 記得只用訓練集的統計量fit，然後transform訓練集和測試集

# 你的程式碼：

---
### 3. 任務1：訓練至少5種不同類型的模型（30分）

**任務要求**：
- 訓練**至少5種**不同類型的回歸模型
- 建議模型類型：
  - 線性模型：Ridge, Lasso, ElasticNet
  - 樹模型：DecisionTree, RandomForest, GradientBoosting, AdaBoost
  - 其他：SVR, KNN
- 對每個模型進行以下評估：
  - **訓練集評估**：R², MAE, RMSE
  - **測試集評估**：R², MAE, RMSE
  - **交叉驗證**：5-fold CV 平均 R²
- 將所有結果整理成表格，方便比較

**評分標準**：
- 訓練5種以上模型（10分）
- 完整的評估指標（10分）
- 結果呈現清晰（表格格式）（10分）

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 請在此訓練至少5種模型並進行評估
# ========================================

# 提示：
# 1. 建立模型字典，包含至少5種模型
# 2. 對每個模型進行訓練與預測
# 3. 計算評估指標（R², MAE, RMSE）
# 4. 進行5-fold交叉驗證
# 5. 將結果整理成DataFrame表格

# 你的程式碼：

---
### 4. 任務2：超參數調整（25分）

**任務要求**：
- 選擇**2個模型**進行超參數調整
- **模型1**：使用 **Grid Search** 進行超參數搜索
- **模型2**：使用 **Bayesian Optimization** 進行超參數搜索
- 比較調整前後的模型性能
- 建議選擇：
  - Grid Search：RandomForest 或 GradientBoosting（參數組合較少）
  - Bayesian Optimization：SVR 或 GradientBoosting（連續參數空間）

**評分標準**：
- Grid Search 實作正確（10分）
- Bayesian Optimization 實作正確（10分）
- 調整前後性能比較分析（5分）

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 請在此進行超參數調整
# ========================================

# 提示：
# 1. 模型1：使用 GridSearchCV 進行超參數搜索
# 2. 模型2：使用 BayesSearchCV 進行超參數搜索
# 3. 比較調整前後的性能差異
# 4. 輸出最佳參數與性能提升幅度

# 你的程式碼：

---
### 5. 任務3：學習曲線與驗證曲線（20分）

**任務要求**：
- 選擇**2個模型**（建議：表現最好的模型 + 表現較差的模型）
- 繪製**學習曲線 (Learning Curve)**：
  - X軸：訓練樣本數量
  - Y軸：R² score
  - 包含訓練集與驗證集曲線
  - 診斷：是否存在過擬合或欠擬合？
- 繪製**驗證曲線 (Validation Curve)**：
  - 選擇一個關鍵超參數（如 max_depth, n_estimators, alpha）
  - X軸：超參數值
  - Y軸：R² score
  - 包含訓練集與驗證集曲線
  - 找出最佳超參數範圍

**評分標準**：
- 學習曲線繪製正確（8分）
- 驗證曲線繪製正確（8分）
- 模型診斷分析（4分）

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 請在此繪製學習曲線與驗證曲線
# ========================================

# 提示：
# 1. 使用 learning_curve() 繪製學習曲線
# 2. 使用 validation_curve() 繪製驗證曲線
# 3. 分析模型是否過擬合或欠擬合
# 4. 找出最佳超參數範圍

# 你的程式碼：

---
### 6. 任務4：模型選擇報告（15分）

**任務要求**：
請根據以上所有實驗結果，撰寫一份**模型選擇報告**，內容包括：

1. **模型性能比較摘要**：
   - 哪個模型在測試集上表現最好？（R², MAE, RMSE）
   - 哪個模型的交叉驗證分數最高？
   
2. **超參數調整效果分析**：
   - Grid Search 和 Bayesian Optimization 哪個更有效？
   - 超參數調整對模型性能的提升幅度如何？
   
3. **模型診斷結果**：
   - 學習曲線顯示哪些模型存在過擬合/欠擬合？
   - 驗證曲線建議的最佳超參數範圍是多少？
   
4. **最終選擇與理由**：
   - 你選擇哪個模型作為最終模型？
   - 理由為何？（考慮性能、穩定性、計算成本、可解釋性等）

**評分標準**：
- 分析邏輯清晰（5分）
- 結論有數據支撐（5分）
- 考慮因素全面（5分）

請在下方 Markdown cell 中撰寫你的報告：

**模型選擇報告**

---

#### 1. 模型性能比較摘要
（請填寫你的分析）

#### 2. 超參數調整效果分析
（請填寫你的分析）

#### 3. 模型診斷結果
（請填寫你的分析）

#### 4. 最終選擇與理由
（請填寫你的選擇與理由）

---
### 7. 任務5：特徵重要性分析（10分）

**任務要求**：
- 對**最佳模型**進行特徵重要性分析
- 使用以下至少**2種方法**：
  - **內建特徵重要性**（適用於樹模型：`feature_importances_`）
  - **Permutation Importance**（適用於所有模型）
  - **部分依賴圖 (PDP)**（選擇前3個重要特徵）
- 從化工角度解釋：
  - 哪些特徵對轉化率影響最大？
  - 是否符合化工原理？（如溫度、催化劑負載量、比表面積等）

**評分標準**：
- 特徵重要性分析正確（5分）
- 繪製清晰的視覺化圖表（3分）
- 化工意義解釋（2分）

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 請在此進行特徵重要性分析
# ========================================

# 提示：
# 1. 使用內建特徵重要性（樹模型）或係數（線性模型）
# 2. 使用 permutation_importance() 進行排列重要性分析
# 3. 使用 PartialDependenceDisplay 繪製部分依賴圖
# 4. 從化工角度解釋結果

# 你的程式碼：

---
## **II. 額外加分作業 (自由繳交)**
- 學生可自由選擇是否繳交加分作業. (下週上課前完成email電子檔)
- 分數會加在最後總成績上, 每份作業加0.1 ~ 1.0分.
- 分數加的不多, 真的不一定要交加分作業, 正常出席上課做好期末報告即可.
- 加分作業不一定要全部都做完, 想繳交至少完成其中一項實驗即可.
- 請務必自行完成! 你可以問AI, 問同學, 問學長姊, 甚至參考以前別人的作業, 但一定要自行[吸收][執行][整理]結果!
- 不要貼AI的答案寄給老師看, 你自己看就好.
- 如果系統自動比對發現作業內容(與他人雷同, 原創性低, 抄襲比例過高), 作業的分數會0分.
- 如果你直接100%複製別人的作業繳交, 你會直接被當掉(提供作業給別人複製的也一樣).
- 老師看你作業要花時間, 真的有心想寫加分作業, 請好好自己做.

---

### 加分題1：集成學習方法比較（+5分）

**任務要求**：
- 實作**Voting Regressor**（投票回歸器）：
  - 結合至少3個不同類型的基模型
  - 嘗試 hard voting 和 soft voting
- 實作**Stacking Regressor**（堆疊回歸器）：
  - 使用至少3個基模型
  - 選擇一個元模型（Meta-model）
- 比較集成方法與單一模型的性能差異

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 額外加分題1 - 集成學習方法
# ========================================

# 你的程式碼（選做）：

---
### 加分題2：特徵工程實驗（+5分）

**任務要求**：
- 對原始數據進行特徵工程：
  - 新增交互特徵（如 Temperature × Catalyst_Loading）
  - 新增多項式特徵（如 Temperature²）
  - 新增領域知識特徵（如 Arrhenius 溫度項： $e^{-\frac{E_a}{RT}}$ ）
- 重新訓練模型並比較性能提升
- 分析哪些特徵工程方法最有效

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 額外加分題2 - 特徵工程實驗
# ========================================

# 你的程式碼（選做）：

---
### 加分題3：殘差分析與模型診斷（+5分）

**任務要求**：
- 對最佳模型進行殘差分析：
  - 繪製**殘差圖 (Residual Plot)**：預測值 vs. 殘差
  - 繪製**殘差直方圖**：檢查是否符合常態分佈
  - 繪製**Q-Q圖 (Quantile-Quantile Plot)**：檢查殘差常態性
- 診斷模型是否存在：
  - 異方差性（Heteroscedasticity）
  - 系統性偏差（Systematic Bias）
  - 異常值（Outliers）

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 額外加分題3 - 殘差分析
# ========================================

# 你的程式碼（選做）：

---
### 加分題4：跨資料集驗證（+5分）

**任務要求**：
- 生成**第二個催化劑數據集**（不同的數據分佈）：
  - 調整溫度範圍（如 250-450°C）
  - 調整催化劑負載量範圍（如 1.0-10.0 wt%）
  - 添加更大的噪聲（模擬不同實驗條件）
- 使用原始數據集訓練的最佳模型預測新數據集
- 評估**模型泛化能力 (Generalization)**：
  - 新數據集的預測性能如何？
  - 是否存在明顯的性能下降？
  - 分析原因並提出改進建議

請在下方完成程式碼：

In [ ]:
# ========================================
# TODO: 額外加分題4 - 跨資料集驗證
# ========================================

# 你的程式碼（選做）：

---
### **💭 思考題**

1. 在本次作業中，我們使用了 R², MAE, RMSE 三種評估指標。請回答：這三種指標各自的優缺點是什麼？在催化劑性能預測場景中，哪個指標最重要？為什麼？

2. 從學習曲線和驗證曲線中，你如何判斷一個模型是否存在過擬合或欠擬合？過擬合和欠擬合的典型特徵各是什麼？在本次作業中，哪些模型可能存在這些問題？你會如何改善？

3. Grid Search 和 Bayesian Optimization 是兩種常見的超參數調整方法。請比較兩者的優勢與限制，並說明在什麼情況下應該選擇哪一種方法？

4. 從特徵重要性分析中，你發現哪些特徵對轉化率影響最大？這些結果是否符合化工原理（如 Arrhenius 定律、催化劑活性理論）？如果結果與理論不符，可能的原因是什麼？

5. 在模型選擇時，我們需要在可解釋性與預測性能之間取得平衡。哪些模型具有較高的可解釋性？哪些模型是"黑箱"？在化工應用中，可解釋性為什麼重要？

6. 為什麼交叉驗證比單次訓練/測試分割更可靠？5-fold 是最佳選擇嗎？如果數據量很大（如 100萬筆），你會如何調整？在時間序列數據中，交叉驗證需要注意什麼？

7. 假設你選定的最佳模型要部署到實際化工廠中，用於線上預測催化劑性能。在部署前，你需要考慮哪些因素？如何監控模型在生產環境中的性能？如果模型性能下降，你會如何應對？

---
# **想對老師說的話**